In [2]:
import xml.etree.ElementTree as ET
from xopen import xopen
import gzip
import polars as pl

In [3]:
PLAN_PATH = "/Users/andre/Desktop/Cergy/MATSim/matsim-berlin/berlin-v6.4.output_plans.xml.gz"
EVENTS_PATH = "/Users/andre/Desktop/Cergy/MATSim/matsim-berlin/berlin-v6.4.output_events.xml.gz"
OUTPUT_DIR = "/Users/andre/Desktop/Cergy/Python_Scripts/runs/fixed_10pct/output/"

EXP_PLANS_PATH = 'berlin-v6.4.output_experienced_plans.xml.gz'

In [10]:
TARGET_VEHICLE = 'bb_0047e8cf_car'

In [11]:
MAX_MATCHES = 1000

matching_lines = []


with gzip.open(PLAN_PATH, 'rt', encoding='utf-8') as f:
    for line in f:
        if f'vehicle="{TARGET_VEHICLE}"' in line:
            matching_lines.append(line.strip())
            if len(matching_lines) >= MAX_MATCHES:
                break


print(f"Nb of lines: {len(matching_lines)}")

Nb of lines: 276


In [12]:
matching_lines

['<event time="28318.0" type="PersonEntersVehicle" person="bb_0047e8cf" vehicle="bb_0047e8cf_car"  />',
 '<event time="28373.0" type="vehicle enters traffic" person="bb_0047e8cf" link="-62237610#1" vehicle="bb_0047e8cf_car" networkMode="car" relativePosition="1.0"  />',
 '<event time="28382.0" type="left link" link="-62237610#1" vehicle="bb_0047e8cf_car"  />',
 '<event time="28382.0" type="entered link" link="-118490073#0" vehicle="bb_0047e8cf_car"  />',
 '<event time="28391.0" type="left link" link="-118490073#0" vehicle="bb_0047e8cf_car"  />',
 '<event time="28391.0" type="entered link" link="-118490076#3" vehicle="bb_0047e8cf_car"  />',
 '<event time="28402.0" type="left link" link="-118490076#3" vehicle="bb_0047e8cf_car"  />',
 '<event time="28402.0" type="entered link" link="-118490076#2" vehicle="bb_0047e8cf_car"  />',
 '<event time="28407.0" type="left link" link="-118490076#2" vehicle="bb_0047e8cf_car"  />',
 '<event time="28407.0" type="entered link" link="-118490076#1" vehicl

In [ ]:
def get_passages():
    records = []

    with gzip.open(EVENTS_PATH, 'rt', encoding='utf-8') as f:
        for event, elem in ET.iterparse(f, events=("end",)):
            if elem.tag == "event":
                event_type = elem.attrib.get("type")
                if event_type in ["entered link", "left link"]:
                    time = float(elem.attrib.get("time", 0))
                    records.append({
                        "time": time,
                        "link_id": elem.attrib.get("link"),
                        "vehicle": elem.attrib.get("vehicle"), 
                        "event_type": event_type
                        })

                if event_type in ["departure", "arrival"]:
                    time = float(elem.attrib.get("time", 0))
                    records.append({
                        "time": time,
                        "link_id": elem.attrib.get("link"),
                        "vehicle": elem.attrib.get("person"), 
                        "event_type": event_type
                    })
            elem.clear()
            
    # Convertir a Polars
    passages_df = pl.DataFrame(records)
    
    return passages_df

In [ ]:
passages_df = get_passages()

In [ ]:
passages_df.write_parquet(OUTPUT_DIR + "passages.parquet")